In [1]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

import cpi
cpi.update()
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'cpi'

In [146]:
train=pd.read_csv('Data/train.csv')

In [147]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [148]:
df=train[['LotArea','BldgType','HouseStyle','YearBuilt','FullBath','HalfBath','BedroomAbvGr','YrSold','SalePrice']].copy()
df.head()

,LotArea,BldgType,HouseStyle,YearBuilt,FullBath,HalfBath,BedroomAbvGr,YrSold,SalePrice
0,8450,1Fam,2Story,2003,2,1,3,2008,208500
1,9600,1Fam,1Story,1976,2,0,3,2007,181500
2,11250,1Fam,2Story,2001,2,1,3,2008,223500
3,9550,1Fam,2Story,1915,1,0,3,2006,140000
4,14260,1Fam,2Story,2000,2,1,4,2008,250000


In [149]:
# use CPI to convert data
df['ADJUSTED_SalesPrice'] = df.apply(lambda x: cpi.inflate(x.SalePrice, x.YrSold), axis=1)
df.head

<bound method NDFrame.head of       LotArea BldgType HouseStyle  YearBuilt  FullBath  HalfBath  \
0        8450     1Fam     2Story       2003         2         1   
1        9600     1Fam     1Story       1976         2         0   
2       11250     1Fam     2Story       2001         2         1   
3        9550     1Fam     2Story       1915         1         0   
4       14260     1Fam     2Story       2000         2         1   
...       ...      ...        ...        ...       ...       ...   
1455     7917     1Fam     2Story       1999         2         1   
1456    13175     1Fam     1Story       1978         2         0   
1457     9042     1Fam     2Story       1941         2         0   
1458     9717     1Fam     1Story       1950         1         0   
1459     9937     1Fam     1Story       1965         1         1   

      BedroomAbvGr  YrSold  SalePrice  ADJUSTED_SalesPrice  
0                3    2008     208500        243172.689187  
1                3    2007     

In [150]:
df.shape

(1460, 10)

In [151]:
# removing outlier
df = df[df.LotArea <= 50000].copy()

In [152]:
df.shape

(1449, 10)

In [153]:
df.columns

Index(['LotArea', 'BldgType', 'HouseStyle', 'YearBuilt', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'YrSold', 'SalePrice',
       'ADJUSTED_SalesPrice'],
      dtype='object')

In [154]:
# split X and Y
X=df[['LotArea','BldgType','HouseStyle','YearBuilt','FullBath','HalfBath','BedroomAbvGr']].copy() # independent variables
Y = df['ADJUSTED_SalesPrice'].copy() # dependent variable

In [155]:
one_hot_df = pd.get_dummies(X[['BldgType','HouseStyle']]) #
X = X.drop(['BldgType','HouseStyle'], axis=1) # Drop column as it is now encoded
X = X.join(one_hot_df) # Join the encoded df
print(X.columns)
X.tail()

Index(['LotArea', 'YearBuilt', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf',
       'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf',
       'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl'],
      dtype='object')


,LotArea,YearBuilt,FullBath,HalfBath,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
1455,7917,1999,2,1,3,1,0,0,0,0,0,0,0,0,0,1,0,0
1456,13175,1978,2,0,3,1,0,0,0,0,0,0,1,0,0,0,0,0
1457,9042,1941,2,0,4,1,0,0,0,0,0,0,0,0,0,1,0,0
1458,9717,1950,1,0,2,1,0,0,0,0,0,0,1,0,0,0,0,0
1459,9937,1965,1,1,3,1,0,0,0,0,0,0,1,0,0,0,0,0


In [156]:
# split data into 70 train and 30 test
X_train_70, X_test_30, Y_train_70, Y_test_30 = train_test_split(X,Y, test_size=0.3, random_state=0)

In [157]:
# split data into 80 train and 20 test
X_train_80, X_test_20, Y_train_80, Y_test_20 = train_test_split(X,Y, test_size=0.2, random_state=0)

## statsmodels ##

In [158]:
# Using 80/20 Split

import statsmodels.api as sm
import math

X_train_80_s = sm.add_constant(X_train_80) # required if a value for alpha is expected
est = sm.OLS(Y_train_80,X_train_80_s).fit() # fit model
predictions = est.predict() # get predicted values
print(est.summary())
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))


                             OLS Regression Results                            
Dep. Variable:     ADJUSTED_SalesPrice   R-squared:                       0.537
Model:                             OLS   Adj. R-squared:                  0.531
Method:                  Least Squares   F-statistic:                     82.82
Date:                 Tue, 06 Jul 2021   Prob (F-statistic):          5.25e-178
Time:                         07:17:12   Log-Likelihood:                -14499.
No. Observations:                 1159   AIC:                         2.903e+04
Df Residuals:                     1142   BIC:                         2.912e+04
Df Model:                           16                                         
Covariance Type:             nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.624e+06

In [159]:
# Using 70/30 Split

import statsmodels.api as sm
import math

X_train_70_s = sm.add_constant(X_train_70) # required if a value for alpha is expected
est = sm.OLS(Y_train_70,X_train_70_s).fit() # fit model
predictions = est.predict() # get predicted values
print(est.summary())
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))


                             OLS Regression Results                            
Dep. Variable:     ADJUSTED_SalesPrice   R-squared:                       0.524
Model:                             OLS   Adj. R-squared:                  0.516
Method:                  Least Squares   F-statistic:                     68.56
Date:                 Tue, 06 Jul 2021   Prob (F-statistic):          3.87e-148
Time:                         07:17:12   Log-Likelihood:                -12693.
No. Observations:                 1014   AIC:                         2.542e+04
Df Residuals:                      997   BIC:                         2.550e+04
Df Model:                           16                                         
Covariance Type:             nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.582e+06

# Prediction Part....Stat....need to continue

# Starting Ridge regression with 10 KNN

In [160]:
# Standardize data
#from sklearn.preprocessing import StandardScaler

#std_scaler = StandardScaler()
#X_train_70 = std_scaler.fit_transform(X_train_70)
#X_train_80 = std_scaler.fit_transform(X_train_80)
#X_test_30 = std_scaler.transform(X_test_30) 
#X_test_20 =std_scaler.transform(X_test_20)

In [161]:
# using 70/30 split

from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


model = Ridge(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train_70, Y_train_70, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE on 70 percent split: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE on 70 percent split: 46172.194 (4233.413)


In [162]:
# using 80/20 split

from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


model = Ridge(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train_80, Y_train_80, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE on 80 percent split: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE on 80 percent split: 45662.698 (4746.063)
